In [56]:
# Library
import tensorflow as tf
import tensorflow_transform as tft
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

### Set Variable

In [57]:
PIPELINE_NAME = "heart-pipeline"
SCHEMA_PIPELINE_NAME = "heart-tfdv-schema"
PIPELINE_ROOT = os.path.join('dewi_karimah-pipeline', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)


In [58]:
DATA_ROOT = "data"

In [59]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

### Data Ingestion

In [60]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [61]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Data Validation

summary statistic

In [62]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [63]:
interactive_context.show(statistics_gen.outputs["statistics"])

data schema

In [64]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [65]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,-
'ca',INT,required,,-
'chol',INT,required,,-
'cp',INT,required,,-
'exang',INT,required,,-
'fbs',INT,required,,-
'oldpeak',FLOAT,required,,-
'restecg',INT,required,,-
'sex',INT,required,,-


anomali dataset

In [66]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [67]:
interactive_context.show(example_validator.outputs['anomalies'])

### Data Preprocessing

In [68]:
TRANSFORM_MODULE_FILE = "heart_transform.py"

In [69]:
%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow_transform as tft

LABEL_KEY = "target"
FEATURE_KEYS = "age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalachh", "exang", "oldpeak", "slope", "ca", "thal"
def transformed_name(key):
     return key + "_xf"
def preprocessing_fn(inputs):
    
    outputs = {
        transformed_name(key): tft.scale_to_0_1(inputs[key])
        for key in FEATURE_KEYS
    }
    
    outputs[transformed_name(LABEL_KEY)] = inputs[LABEL_KEY]
    
    return outputs

Overwriting heart_transform.py


In [70]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

INFO:tensorflow:Assets written to: dewi_karimah-pipeline\heart-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\c7fd55be6c2a4cad88d47a46ad4a32ad\assets


INFO:tensorflow:Assets written to: dewi_karimah-pipeline\heart-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\c7fd55be6c2a4cad88d47a46ad4a32ad\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Tuner

In [71]:
TUNER_MODULE_FILE = "heart_tuner.py"

In [ ]:
%%writefile {TUNER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers
import os  

from kerastuner.engine import base_tuner 
from typing import NamedTuple, Dict, Text, Any 
from tfx.components.trainer.fn_args_utils import FnArgs
from tfx.components import Tuner
import kerastuner as kt

LABEL_KEY = "target"
FEATURE_KEYS = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalachh", "exang", "oldpeak", "slope", "ca", "thal"]

def transformed_name(key):
    return key + "_xf"

def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


TunerFnResult = NamedTuple('TunerFnResult', [('tuner', base_tuner.BaseTuner),
                                            ('fit_kwargs', Dict[Text,Any])])

def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=100)->tf.data.Dataset:
    
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY)
    )
    return dataset

def model_builder(hp):
    num_features = len(FEATURE_KEYS)
    transformed_FEATURE_KEYS = [transformed_name(key) for key in FEATURE_KEYS]
    inputs = [tf.keras.Input(shape=(1,), name=f) for f in transformed_FEATURE_KEYS]
    x = layers.concatenate(inputs)
    
    hp_units = hp.Int('units', min_value=8, max_value=48, step=8)
    x = layers.Dense(hp_units, activation="relu")(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    hp_learning_rate = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    
    model.summary()
    return model

def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    tuner = kt.RandomSearch(
        model_builder,
        objective='val_binary_accuracy',
        max_trials=5,
        directory=fn_args.working_dir,
        project_name='heart_tuning'
    )
    
    train_set = input_fn(fn_args.train_files, tf_transform_output, num_epochs=10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, num_epochs=10)
    
    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            "x": train_set,
            "validation_data": val_set,
            "epochs": 25
        }
    )


Overwriting heart_tuner.py


In [73]:
from tfx.proto import trainer_pb2
 
tuner  = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)
interactive_context.run(tuner)

Trial 5 Complete [00h 00m 55s]
val_binary_accuracy: 0.8283378481864929

Best val_binary_accuracy So Far: 0.8419618606567383
Total elapsed time: 00h 04m 03s
Results summary
Results in dewi_karimah-pipeline\heart-pipeline\.temp\6\heart_tuning
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
units: 24
learning_rate: 0.01
Score: 0.8419618606567383

Trial 3 summary
Hyperparameters:
units: 40
learning_rate: 0.0001
Score: 0.8392370343208313

Trial 2 summary
Hyperparameters:
units: 16
learning_rate: 0.0001
Score: 0.8337874412536621

Trial 0 summary
Hyperparameters:
units: 48
learning_rate: 0.0001
Score: 0.8310626745223999

Trial 4 summary
Hyperparameters:
units: 16
learning_rate: 0.01
Score: 0.8283378481864929


ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Trainer

In [74]:
TRAINER_MODULE_FILE = "heart_trainer.py"

In [ ]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs
from kerastuner.engine.hyperparameters import HyperParameters
import os

LABEL_KEY = "target"
FEATURE_KEYS = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalachh", "exang", "oldpeak", "slope", "ca", "thal"]

def transformed_name(key):
    return key + "_xf"

def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern,
             tf_transform_output,
             num_epochs=None,
             batch_size=128) -> tf.data.Dataset:
    transform_feature_spec = tf_transform_output.transformed_feature_spec().copy()
    return tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY)
    )

def model_builder(hp):
    num_features = len(FEATURE_KEYS)
    transformed_FEATURE_KEYS = [transformed_name(key) for key in FEATURE_KEYS]
    inputs = [tf.keras.Input(shape=(1,), name=f) for f in transformed_FEATURE_KEYS]
    x = layers.concatenate(inputs)

    # Mengambil nilai parameter terbaik dari tuner
    hp_units = hp['values']['units'] 
    hp_learning_rate = hp['values']['learning_rate']

    x = layers.Dense(hp_units, activation="relu")(x)

    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)


    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

    model.summary()
    return model

def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        
        feature_spec = tf_transform_output.raw_feature_spec()
        
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        
        return model(transformed_features)
        
    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    hp = fn_args.hyperparameters
    model = model_builder(hp)

    train_dataset = input_fn(fn_args.train_files, tf_transform_output, num_epochs=10)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output, num_epochs=10)

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=5)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)
    

    model.fit(
        train_dataset,
        epochs=25,
        validation_data=eval_dataset,
        callbacks=[tensorboard_callback, es, mc]
    )

    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }

    # Save the model
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)


Overwriting heart_trainer.py


In [76]:
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)
interactive_context.run(trainer)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 age_xf (InputLayer)         [(None, 1)]                  0         []                            
                                                                                                  
 sex_xf (InputLayer)         [(None, 1)]                  0         []                            
                                                                                                  
 cp_xf (InputLayer)          [(None, 1)]                  0         []                            
                                                                                                  
 trestbps_xf (InputLayer)    [(None, 1)]                  0         []                            
                                                                                            

INFO:tensorflow:Assets written to: dewi_karimah-pipeline\heart-pipeline\Trainer\model\7\Format-Serving\assets


119/119 [==============================] - 17s 74ms/step - loss: 0.5228 - binary_accuracy: 0.7371 - val_loss: 0.4856 - val_binary_accuracy: 0.7902
Epoch 2/25
117/119 [============================>.] - ETA: 0s - loss: 0.3904 - binary_accuracy: 0.8154
Epoch 2: val_binary_accuracy did not improve from 0.79019
119/119 [==============================] - 3s 23ms/step - loss: 0.3904 - binary_accuracy: 0.8153 - val_loss: 0.4936 - val_binary_accuracy: 0.7657
Epoch 3/25
119/119 [==============================] - ETA: 0s - loss: 0.3189 - binary_accuracy: 0.8540
Epoch 3: val_binary_accuracy improved from 0.79019 to 0.82016, saving model to dewi_karimah-pipeline\heart-pipeline\Trainer\model\7\Format-Serving
INFO:tensorflow:Assets written to: dewi_karimah-pipeline\heart-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: dewi_karimah-pipeline\heart-pipeline\Trainer\model\7\Format-Serving\assets


119/119 [==============================] - 5s 44ms/step - loss: 0.3189 - binary_accuracy: 0.8540 - val_loss: 0.4798 - val_binary_accuracy: 0.8202
Epoch 4/25
116/119 [============================>.] - ETA: 0s - loss: 0.2778 - binary_accuracy: 0.8758
Epoch 4: val_binary_accuracy did not improve from 0.82016
119/119 [==============================] - 2s 15ms/step - loss: 0.2774 - binary_accuracy: 0.8761 - val_loss: 0.5001 - val_binary_accuracy: 0.8120
Epoch 5/25
118/119 [============================>.] - ETA: 0s - loss: 0.2429 - binary_accuracy: 0.8948
Epoch 5: val_binary_accuracy did not improve from 0.82016
119/119 [==============================] - 2s 15ms/step - loss: 0.2431 - binary_accuracy: 0.8945 - val_loss: 0.5293 - val_binary_accuracy: 0.7711
Epoch 6/25
116/119 [============================>.] - ETA: 0s - loss: 0.2184 - binary_accuracy: 0.9045
Epoch 6: val_binary_accuracy did not improve from 0.82016
119/119 [==============================] - 2s 19ms/step - loss: 0.2183 - binary

...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: dewi_karimah-pipeline\heart-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: dewi_karimah-pipeline\heart-pipeline\Trainer\model\7\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Analisis dan Validasi Model

Resolver

In [77]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [78]:
import tensorflow_model_analysis as tfma 
 
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='target')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
 
)

Evaluator

In [79]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [80]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

Pusher

In [81]:
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 

pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/heart_disease_prediction'))

)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)